In [ ]:
#!pip install --quiet langchain-anthropic langchain-neo4j cyVer

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from tqdm import tqdm
import pandas as pd

from langchain_anthropic import ChatAnthropic
from utils import (
    _value_sanitize,
    extract_json_from_markdown,
    sampling_query,
    validate_cypher,
    process_database,
    process_all_examples_with_limit,
    convert_datetime
)

# Generate dataset

In [ ]:
# LLM selection
model = ChatAnthropic(model='claude-opus-4-20250514')

# Database selection (for demo database)
db_url = "neo4j+s://demo.neo4jlabs.com"
databases = [
    "companies",
    "twitch", 
    "network",
    "northwind",
    "ClinicalKnowledgeGraph"
]


In [ ]:
iterations_per_database = 2 # * 24 records per generation pass
output = []

for credential in tqdm(databases, desc="Processing databases"):
    database_records = process_database(
        credential, db_url, model, iterations_per_database
    )
    output.extend(database_records)

# Generate text answers

In [ ]:
qa_model = ChatAnthropic(model='claude-3-5-haiku-latest')

In [ ]:
# Generate text-based answers
await process_all_examples_with_limit(output, qa_model)

In [ ]:
df = pd.DataFrame.from_records(output)
df.head(5)

In [ ]:
with open("generated_dataset.json", "w") as f:
    json.dump(output, f, indent=2, default=convert_datetime)